In [118]:
import pandas as pd
import sqlalchemy
import ssl
import spacy
import nltk.corpus
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
from spacytextblob.spacytextblob import SpacyTextBlob

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords



conn_string = # "redacted due to credential exposure" 
engine = sqlalchemy.create_engine(conn_string)

sql_data = pd.read_sql_table('youtube_metrics',engine)
stop_words = stopwords.words('english')

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amalkadri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
sql_data
metrics = sql_data.iloc[0].to_dict()
analysis = {
    'videoID' : metrics['videoID'],
    'views_count' : int(metrics['views']),
    'likes': int(metrics['likes']),
    'comments_' : int(metrics['comments']),
    'length_' : metrics['length'],
    'like_ratios' : int(metrics['likes'])/int(metrics['views']),
    'comment_ratio' : int(metrics['comments'])/int(metrics['views'])
}
analysis

def sql_comments(video):
    """gets comments from comments database"""
    sql_comm = pd.read_sql(f"""select * from youtube_comments where "videoID" = '{video}'""", engine).drop_duplicates()
    return sql_comm

sql_comments("KjPDJLlzirM")

,index,videoID,comment
0,3,KjPDJLlzirM,"You always publish the best videos, I have a q..."
1,1,KjPDJLlzirM,"Great videos. They've helped me paint my 3,000..."
2,2,KjPDJLlzirM,"Great Job Chris! Very good work, thanks for sh..."
3,4,KjPDJLlzirM,I would love to see how you mix the lacquer.
4,0,KjPDJLlzirM,Great video! Would be cool to see more on your...
5,5,KjPDJLlzirM,"I love your channel good job,just a quick ques..."
6,6,KjPDJLlzirM,"Chris,\n\nI am looking at spraying some Built-..."
7,7,KjPDJLlzirM,Excellent video. Would also like to how you cl...
8,8,KjPDJLlzirM,Is there any difference in the cleaning proces...
9,9,KjPDJLlzirM,What sizes of FFLP fine finish do you like to ...


In [3]:
comments_df = pd.read_sql_table('youtube_comments',engine)

In [4]:
comments_df


,index,videoID,comment
0,0,SwSbnmqk3zY,"Hi Thoufiq, this is very impressive! I’m watch..."
1,1,SwSbnmqk3zY,"Hi Thoufiq, Thanks for sharing this informativ..."
2,2,SwSbnmqk3zY,"Hi Thoufiq, excellent video absolutely loved i..."
3,3,SwSbnmqk3zY,"Thanks, Brother. This was a bit intricate for ..."
4,4,SwSbnmqk3zY,One of the best Analysis video I have seen so ...
...,...,...,...
124983,5,pJiYw2pUceg,Gorgeous
124984,6,pJiYw2pUceg,Sick video. This is the best Bloody Mary mix
124985,7,pJiYw2pUceg,sweet😋😚👍
124986,8,pJiYw2pUceg,this❤❤


In [5]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

In [26]:
comm_analysis = comments_df[['videoID','comment']].drop_duplicates().head(100)
comm_analysis['comment'] = comm_analysis.groupby(['videoID'])['comment'].transform(lambda x : ' '.join(x))
comm_analysis = comm_analysis.drop_duplicates().reset_index()[['videoID','comment']]
comm_analysis

,videoID,comment
0,SwSbnmqk3zY,"Hi Thoufiq, this is very impressive! I’m watch..."
1,B_xNI6tuOeI,Very Impressive finishing work. Nice to see th...
2,vGJL0BgVMOw,perfect taste combination... gonna try this re...
3,ianApRDflh0,Here’s the potato harvest! https://youtu.be/UK...
4,8nQUQJKnX0E,Good
5,K4a3dTScBlg,Having a toilet was must for our camper van an...
6,pb4xXXEA8zk,Girl!! You are on a serious streak with these ...


In [27]:
comm_analysis['clean'] =  comm_analysis['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
comm_analysis['clean'] = comm_analysis['clean'].transform(lambda txt: clean_text_round1(txt)).transform(lambda txt: clean_text_round2(txt))

In [19]:
text = comm_analysis['clean'][0]
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("spacytextblob")
nlp(text)
nlp(text)._.blob.polarity
get_polarity(text)
# print(doc._.blob.polarity)
# print(doc._.blob.subjectivity)


0.30122663921444404

In [46]:
def get_polarity(txt):
    return nlp(txt)._.blob.polarity
polarity_getter = get_polarity
def get_subjectivity(txt):
    return nlp(txt)._.blob.subjectivity
subjectivity_getter = get_subjectivity

def get_sentiment(txt):
    doc = nlp(txt)
    sentiment_list = [doc._.blob.polarity,doc._.blob.subjectivity]
    return sentiment_list
sentiment_getter = get_sentiment

In [45]:
final_text_analysis = comm_analysis
final_text_analysis['polarity'] = final_text_analysis['clean'].apply(polarity_getter)
final_text_analysis['subjectivity'] = final_text_analysis['clean'].apply(subjectivity_getter)
final_text_analysis

,videoID,comment,clean,polarity,subjectivity
0,SwSbnmqk3zY,"Hi Thoufiq, this is very impressive! I’m watch...",hi thoufiq impressive im watching purely curio...,0.301227,0.545183
1,B_xNI6tuOeI,Very Impressive finishing work. Nice to see th...,very impressive finishing work nice see pride ...,0.352873,0.541713
2,vGJL0BgVMOw,perfect taste combination... gonna try this re...,perfect taste combination gonna try recipe 😋😋n...,0.600000,0.920000
3,ianApRDflh0,Here’s the potato harvest! https://youtu.be/UK...,heres potato harvest httpsyoutubeukmsnboeofe i...,0.271048,0.480078
4,8nQUQJKnX0E,Good,good,0.700000,0.600000
5,K4a3dTScBlg,Having a toilet was must for our camper van an...,having toilet must camper van were super happy...,0.237805,0.486941
6,pb4xXXEA8zk,Girl!! You are on a serious streak with these ...,girl you serious streak last few i mean i love...,0.120417,0.537083


In [63]:
final_text_analysis = comm_analysis[['videoID','clean']]
final_text_analysis[['polarity','subjectivity']] = pd.DataFrame(final_text_analysis['clean'].apply(sentiment_getter).to_list(), columns =   ['polarity','subjectivity'])
final_text_analysis

/var/folders/18/lvwyjtw159jgvq01xkgtwzj00000gn/T/ipykernel_25120/3171715115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_text_analysis[['polarity','subjectivity']] = pd.DataFrame(final_text_analysis['clean'].apply(sentiment_getter).to_list(), columns =['polarity','subjectivity'])
/var/folders/18/lvwyjtw159jgvq01xkgtwzj00000gn/T/ipykernel_25120/3171715115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_text_analysis[['polarity','subjectivity']] = pd.DataFrame(final_text_analysis['clean

,videoID,clean,polarity,subjectivity
0,SwSbnmqk3zY,hi thoufiq impressive im watching purely curio...,0.301227,0.545183
1,B_xNI6tuOeI,very impressive finishing work nice see pride ...,0.352873,0.541713
2,vGJL0BgVMOw,perfect taste combination gonna try recipe 😋😋n...,0.600000,0.920000
3,ianApRDflh0,heres potato harvest httpsyoutubeukmsnboeofe i...,0.271048,0.480078
4,8nQUQJKnX0E,good,0.700000,0.600000
5,K4a3dTScBlg,having toilet must camper van were super happy...,0.237805,0.486941
6,pb4xXXEA8zk,girl you serious streak last few i mean i love...,0.120417,0.537083


In [71]:
row_dict = final_text_analysis.iloc[0].to_dict()
row_dict


{'videoID': 'SwSbnmqk3zY',
 'clean': 'hi thoufiq impressive im watching purely curiosity bc ive started python class yet however i would love see sql project ideas maybe data cleaning andor analysis thanks always clear explanations guidance 👍🏻 also glad see youre closing  friend congratulations advance 😀👌🏻🙏🏻 hi thoufiq thanks sharing informative  interesting video i wondering extract actual comments replies instead comment count example thanks advance hi thoufiq excellent video absolutely loved please keep creating many project videos can make short video visualize data using python dynamic radar charts ie player vs player comparisioncompany vs company  filter two competitors compare thanks brother this bit intricate i started learning python data analysis tricks used new i eager learn use create new project based information dont worry keep it bigger ken jee one day stay consistent one best analysis video i seen far amazing content  expecting videos projects  appreciate hard work  hi 